# Segmentation to btrack BATCH PROCESS

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [14]:
import glob
import os
import btrack
print('Btrack version', btrack.__version__)
import napari
print('Napari version', napari.__version__)
import numpy as np

from skimage.io import imread
from natsort import natsorted
from napari_animation import AnimationWidget
from tqdm import tqdm
import tifffile as tiff
from datetime import datetime

### function to load images efficiently
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = imread(filename)
        yield img

Btrack version 0.4.1
Napari version 0.4.7


# Set root folder and find expt IDs

In [4]:
root_folder = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/'
expt_IDs = natsorted([ID for ID in os.listdir(root_folder) if 'MK' in ID])
expt = expt_IDs[0]

# Run batch tracking job

In [ ]:
pos_list = natsorted([pos for pos in os.listdir(os.path.join(root_folder, expt)) if 'Pos' in pos])
for pos in tqdm(pos_list):
    try:

        ### display info
        print(pos, 'starting')

        print('Finding mask files', pos)
        ### find masks and sort numerically
        mask_dir = os.path.join(root_folder,'{}/{}/{}_cp_masks'.format(expt, pos, pos))
        files = natsorted(glob.glob(os.path.join(mask_dir, '*.png')))
        #print(files)
        if len(files) == 0:
            print('No masks found for', pos, ', continuing to next position')
            continue

        print('Filtering mask files', pos)
        ## filter files
        gfp_files = [file for file in files if 'channel001' in file]
        rfp_files = [file for file in files if 'channel002' in file]

        print('Loading mask files', pos)
        ### load images
        gfp_generator = segmentation_generator(gfp_files)
        rfp_generator = segmentation_generator(rfp_files)

        print('Localising objects', pos)
        ### localise objects in images
        gfp_obj_from_generator = btrack.utils.segmentation_to_objects(gfp_generator, properties = ('area', 'major_axis_length'))
        rfp_obj_from_generator = btrack.utils.segmentation_to_objects(rfp_generator, properties = ('area', 'major_axis_length'))

        print('Tracking GFP objects', pos)
        ### track gfp objects
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:

            # configure the tracker using a config file
            tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json')
            tracker.max_search_radius = 10

            # append the objects to be tracked
            #tracker.append(gfp_obj_from_arr) #obj_from_generator)
            tracker.append(gfp_obj_from_generator)

            # set the volume
            tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

            # track them (in interactive mode)
            tracker.track_interactive(step_size=100)

            # generate hypotheses and run the global optimizer
            tracker.optimize()
            
            ## export with guaranteed unique fn 
            date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
            tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'gfp_tracks_{}.h5'.format(date)), obj_type='obj_type_1')

            # get the tracks in a format for napari visualization
            #gfp_data, gfp_properties, gfp_graph = tracker.to_napari(ndim=2)
            #gfp_tracks = tracker.tracks

        print('Tracking RFP objects', pos)
        ### track rfp objects 
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:

            # configure the tracker using a config file
            tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_scribble_sparse.json')
            tracker.max_search_radius = 10

            # append the objects to be tracked
            #tracker.append(rfp_obj_from_arr) #obj_from_generator)
            tracker.append(rfp_obj_from_generator)

            # set the volume
            tracker.volume=((0, 1688),(0, 1352),(-1e5, 1e5))

            # track them (in interactive mode)
            tracker.track_interactive(step_size=100)

            # generate hypotheses and run the global optimizer
            tracker.optimize()
            
            ## export with guaranteed unique fn 
            date = datetime.now().strftime("%Y_%m_%d_%I_%M_%S_%p")
            tracker.export(os.path.join(root_folder, '{}/{}'.format(expt, pos), 'rfp_tracks_{}.h5'.format(date)), obj_type='obj_type_2')

            # get the tracks in a format for napari visualization
            #rfp_data, rfp_properties, rfp_graph = tracker.to_napari(ndim=2)
            #rfp_tracks = tracker.tracks
        print('Finished', pos)
        
    except:
        print(pos, 'Failed to generate tracking file (masks probably missing)')

  0%|          | 0/24 [00:00<?, ?it/s]

Pos0 starting
Finding mask files Pos0


[INFO][2021/05/21 05:11:04 PM] Localizing objects from segmentation...


Filtering mask files Pos0
Loading mask files Pos0
Localising objects Pos0


[INFO][2021/05/21 05:15:02 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/21 05:15:06 PM] ...Found 563356 objects in 1014 frames.
[INFO][2021/05/21 05:15:06 PM] Localizing objects from segmentation...
[INFO][2021/05/21 05:15:28 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/21 05:15:28 PM] ...Found 377 objects in 419 frames.
[INFO][2021/05/21 05:15:28 PM] Loaded btrack: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/05/21 05:15:28 PM] btrack (v0.4.1) library imported
[INFO][2021/05/21 05:15:28 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/21 05:15:28 PM] Starting BayesianTracker session
[INFO][2021/05/21 05:15:28 PM] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/MDCK_config_new.json
[INFO][2021/05/21 05:15:28 PM] Loading motion model: b'MDCK_motion_Kristina'
[INFO][2021/05/21 05:15:28 PM] Setting max XYZ search radius to: 10
[INFO][2021/05/21 05:15:28 PM] Objects are

...

Tracking RFP objects Pos0


[INFO][2021/05/21 05:18:22 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Manasi/MK0000/Pos0/rfp_tracks_2021_05_21_05_18_22_PM.h5
[INFO][2021/05/21 05:18:22 PM] Ending BayesianTracker session
  4%|▍         | 1/24 [07:19<2:48:18, 439.07s/it]

Finished Pos0
Pos1 starting
Finding mask files Pos1
No masks found for Pos1 , continuing to next position
Pos2 starting
Finding mask files Pos2
No masks found for Pos2 , continuing to next position
Pos3 starting
Finding mask files Pos3
No masks found for Pos3 , continuing to next position
Pos4 starting
Finding mask files Pos4
No masks found for Pos4 , continuing to next position
Pos5 starting
Finding mask files Pos5


[INFO][2021/05/21 05:18:23 PM] Localizing objects from segmentation...


Filtering mask files Pos5
Loading mask files Pos5
Localising objects Pos5


# and generate glimpses

(borrowed from other notebook to run sequentially as i want to go to the pub now instead of waiting for this to finish before running glimpse creator ty v much)

In [ ]:
import btrack
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imsave, imshow
from tqdm import tqdm
import numpy as np
from pathlib import Path
from natsort import natsorted
import warnings
warnings.filterwarnings('ignore') ###bc imsave throws up low contrast warnings

In [ ]:
expt_root_folder = root_folder

channels = ['003'] ### bf 000 gfp 001 rfp 002 irfp 003

focal_time_range = (0, 800) ### this particular expt stays in focus in this time range

for pos in pos_list:
    print('starting', pos)
    tracks_path = os.path.join(expt_root_folder,'{}/{}/gfp_tracks.h5'.format(expt, pos))

    with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
        wt_tracks = hdf.tracks

    ### filter tracks that should contain a full cell cycle in (1 timepoint/frame = 4min, cc time approx 200 frames based on quick look)
    long_tracks = [track for track in wt_tracks if len(track) > 200]

    ### filter tracks that spend a lot of time out of focus
    tracks = [track for track in long_tracks if track.t[-1] < focal_time_range[1]]

    ### show number of filtered tracks
    print('number of glimpses to be generated:', len(tracks))

    ### find images and set output
    image_folder = os.path.join(expt_root_folder,expt, pos, pos+'_stacks')
    output_dir = '/home/nathan/data/kraken/pcna/glimpses/{}/{}'.format(expt, pos)

    for channel in channels:
        ### load images
        print('Loading image channel', channel)
        images = imread(os.path.join(image_folder, 'channel{}.tif'.format(channel)))
        print('Image loaded')
        ### create output directory
        output_ch_dir = os.path.join(output_dir, 'channel'+channel)
        Path(output_ch_dir).mkdir(parents=True, exist_ok=True)
        #os.mkdir(output_ch_dir) if os.path.exists(output_ch_dir) == False else print(output_ch_dir, 'already exists')

        for cell in tqdm(tracks):
            try:
                ### get length of track/glimpse 
                length = range(cell.t[0], cell.t[-1])
                ### get the xy coords of track/glimpse 
                y, x = cell.x, cell.y #### WHY IS THE COORDINATES TRANSPOSED???? 

                ### iterate over all frames cropping
                for j, i in enumerate(length): ## need to have something that solves skipped frames # not including the last frame bc list index out of range ie there is one more i than there
                    ### obtain correct frame
                    image = images[i]
                    ### obtain coordinates of bounding box of glimpse (centered on cell)
                    x1, x2, y1, y2 = x[j], x[j]+200, y[j], y[j]+200 #coordinates to crop by, shifted bearing in mind a further coord shift (+100,+100) occurs due to image padding            
                    ### pad image incase cell goes off FOV
                    image = np.pad(image, 100, 'constant', constant_values = 0) 
                    ## glimpse creation by cropping original image
                    glimpse = image[int(x1): int(x2), int(y1): int(y2)]
                    ### fraction of track this frame is at
                    age = round(i/cell.t[-1],3) 
                    fn = 'cell_ID_' + str(cell.ID) + '_channel' + channel + '_t{}_age{}.tif'.format(i, age)
                    ### glimpse output
                    imsave(os.path.join(output_ch_dir, fn), glimpse) 
                print('cell ID',cell.ID, 'glimpse saved')

            except:
                print(cell.ID, 'failed to produce glimpse')
